In [4]:
import numpy as np
import pandas as pd
#import tensorflow as tf

import contractions
import os
from collections import Counter
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
from sklearn.preprocessing  import LabelEncoder
le = LabelEncoder()
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from nltk.corpus import stopwords    
from nltk.tokenize import word_tokenize
from textblob import TextBlob

from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, SpatialDropout1D, MaxPooling1D, Conv1D, Concatenate, Bidirectional, GlobalMaxPool1D, ActivityRegularization, BatchNormalization
from keras.models import Model
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score,confusion_matrix, classification_report

import nltk
# nltk.download('wordnet')
# nltk.download('omw-1.4')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import nlpaug
import nlpaug.augmenter.word as nlpaw

%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
!pip install yfinance
import yfinance as yf

from pandas.tseries.offsets import DateOffset
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

[nltk_data] Downloading package stopwords to C:\Users\Ankit
[nltk_data]     Shah\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
path = '../data/Transcripts'
df = pd.DataFrame()
companies = {'AMZN':'Amazon',
 'AAPL':'Apple',
 'INTC':'Intel',
 'MU':'Micron',
 'GOOGL': 'Alphabet',
 'MSFT': 'Microsoft',
 'AMD':'AMD',
 'ASML':'ASML',
 'NVDA':'NVIDIA',
 'CSCO':'Cisco'}

In [6]:
df = []
tf = 0
totalsummary = ''
totalqna = ''
for comp in os.listdir(path):
    if comp!='.DS_Store':
        company = comp
        files = os.listdir(path+'/'+company)
        for j in files:
            allinfo = []
            with open(path+'/'+company+'/'+j) as f:
                lines = f.readlines()
                summary = []
                qna = []
                i = 0
                while i<len(lines):
                    if lines[i]=='\n':
                        i+=1
                    elif lines[i][:3]=='===':
                        i+=1
                    elif 'operator' in lines[i].lower():
                        i+=3
                    elif lines[i][:3] == '---':
                        i+=1
                    elif 'presentation' in lines[i].lower():
                        i+=2
#                         print(j)
                        while i<len(lines):
#                             print(lines[i])
                            if '===' in lines[i]:
                                break
                            if 'Questions and Answers' in lines[i]:
                                break
                            elif lines[i]=='\n':
                                i+=1
                                continue
                            elif 'operator' in lines[i].lower():
                                i+=3
                            elif '---' in lines[i]:
                                i+=1
                            summary.append(lines[i].rstrip())
                            i+=1
                    
                    elif '[' in lines[i] and companies[company].lower() in lines[i].lower():
                        i+=2
                        while lines[i][:3]!='---':
                            if lines[i]=='\n':
                                i+=1
                                continue
                            summary.append(lines[i].rstrip())
                            i+=1
                    
                    elif 'Questions and Answers' in lines[i]:
                        i+=1
                        while i<len(lines):
                            if lines[i]=='\n':
                                i+=1
                            elif 'Operator' in lines[i]:
                                i+=2
                            elif lines[i][:3] == '---':
                                i+=1
                            elif '[' in lines[i]:
                                i+=2
                            else:
                                qna.append(lines[i].rstrip())
                                i+=1
                    
                    elif '[' in lines[i] and companies[company].lower() not in lines[i].lower():
                        while i<len(lines):
                            if lines[i]=='\n':
                                i+=1
                            elif 'Operator' in lines[i]:
                                i+=3
                            elif lines[i][:3] == '---':
                                i+=1
                            elif '[' in lines[i]:
                                i+=2
                            else:
                                qna.append(lines[i].rstrip())
                                i+=1
                    else:
                        i+=1
                
                if lines[0][0] == 'Q':
                    quarter = lines[0].split(' ')[0]
                    year = lines[0].split(' ')[1]
                else:
                    quarter = lines[1].split(' ')[0]
                    year = lines[1].split(' ')[1]
                cleansummary = []
#                 print(j)
                for i in range(len(summary)):
                    if '--' in summary[i]:
                        i+=1
                    elif 'Operator' in summary[i]:
                        i+=3
                    elif '[' in summary[i]:
#                         print(i, summary[i])
                        i+=1
                    else:
                        cleansummary.append(summary[i])
                summary_para = ' '.join(cleansummary).rstrip()
                qna_para = ' '.join(qna).rstrip()
                totalsummary += summary_para
                totalqna = qna_para
                allinfo.append(quarter)
                allinfo.append(year)
                allinfo.append(companies[company])
                allinfo.append(summary_para)
                allinfo.append(qna_para)
                df.append(allinfo)

In [7]:
earnings_transcript_df = pd.DataFrame(df)
earnings_transcript_df.columns =['Quarter','Year','Company','Summary','Q&A']

In [8]:
earnings_transcript_df

,Quarter,Year,Company,Summary,Q&A
0,Q2,2016,Apple,"Good day, everyone, and welcome to t...",My first question was actually just ...
1,Q1,2016,Apple,I. 1Q16 Business Review (T.C.) 1. Highli...,"<Sync id=""L166""/>Hello. <Sync id=""L..."
2,Q3,2016,Apple,I. Annotation (N.P) 1. Detail: ...,"Thank you very much for the question. Tim, can..."
3,Q4,2016,Apple,I. 4Q16 Business Review (T.C.) 1. Highli...,"Good afternoon and congratulations. Tim, now t..."
4,Q3,2017,Apple,I. 3Q17 Review (T.C.) 1. Highlights: ...,"<Sync id=""L153""time=""00:25:09""/>Luca..."
...,...,...,...,...,...
183,Q1,2020,NVIDIA,I'll now turn the call over to Simona Jankowsk...,"Colette, I was wondering if you coul..."
184,Q3,2020,NVIDIA,I'll now turn the call over to Simona Jankowsk...,"For my first one, you mentioned that..."
185,Q2,2021,NVIDIA,"Simona Jankowski, you may begin your conferenc...",Congratulations on the strong growth and execu...
186,Q4,2020,NVIDIA,"Thank you. Good afternoon, everyone...","I guess on data center, Colette or J..."


In [6]:
words = Counter(totalsummary.split(' '))
words += Counter(totalqna.split(' '))

In [9]:
from readability import Readability
r = Readability(earnings_transcript_df.iloc[0,3])
r.flesch_kincaid()
print(r.flesch())
print(r.gunning_fog())

score: 49.678204038670316, ease: 'difficult', grade_levels: ['college']
score: 14.0770290173987, grade_level: 'college'


In [10]:
def getAvgTxtLen(df):
    avg_length_summary_after_preprocessing = str(df['Summary'].str.len().mean()) 
    avg_length_qna_after_preprocessing = str(df['Q&A'].str.len().mean()) 
    print(avg_length_summary_after_preprocessing)
    print(avg_length_qna_after_preprocessing)

In [11]:
def strip_html_tags(text): 
    soup = BeautifulSoup(text, "html.parser") 
    stripped_text = soup.get_text(separator=" ") 
    return stripped_text 
    
# Make lowercase
earnings_transcript_df["Summary"] = earnings_transcript_df["Summary"].str.lower()
earnings_transcript_df["Q&A"] = earnings_transcript_df["Q&A"].str.lower()

# Remove URLS
earnings_transcript_df['Summary']= earnings_transcript_df['Summary'].str.replace('http(s)?:\/\/.*', '', regex=True)
earnings_transcript_df['Q&A']= earnings_transcript_df['Q&A'].str.replace('http(s)?:\/\/.*', '', regex=True)
print("Remove URLS")
getAvgTxtLen(earnings_transcript_df)

# remove HTML tags
earnings_transcript_df['Summary'] = earnings_transcript_df['Summary'].apply(lambda x: strip_html_tags(x))
earnings_transcript_df['Q&A'] = earnings_transcript_df['Q&A'].apply(lambda x: strip_html_tags(x))
print("remove HTML tags")
getAvgTxtLen(earnings_transcript_df)

# remove extra whitespaces
earnings_transcript_df['Summary'] = earnings_transcript_df['Summary'].replace(r'/\s/\s+/g', ' ', regex=True)
earnings_transcript_df['Q&A'] = earnings_transcript_df['Q&A'].replace(r'/\s/\s+/g', ' ', regex=True)
print("remove extra whitespaces")
getAvgTxtLen(earnings_transcript_df)

# remove contractions
earnings_transcript_df['Summary'] = earnings_transcript_df['Summary'].apply(lambda x: ' '.join([contractions.fix(word) for word in x.split()]))
earnings_transcript_df['Q&A'] = earnings_transcript_df['Q&A'].apply(lambda x: ' '.join([contractions.fix(word) for word in x.split()]))
print("remove contractions")
getAvgTxtLen(earnings_transcript_df)

# Remove characters following digits such as 1st, 2nd or 3rd
earnings_transcript_df['Summary'] = earnings_transcript_df['Summary'].str.replace('\b\d+\w+\b', '', regex=True)
earnings_transcript_df['Q&A'] = earnings_transcript_df['Q&A'].str.replace('\b\d+\w+\b', '', regex=True)
print("Remove characters following digits such as 1st, 2nd or 3rd")
getAvgTxtLen(earnings_transcript_df)

# Replace hyphens with space
earnings_transcript_df['Summary'] = earnings_transcript_df['Summary'].replace('/\b-\b/g', ' ', regex=True)
earnings_transcript_df['Q&A'] = earnings_transcript_df['Q&A'].replace('/\b-\b/g', ' ', regex=True)
print("Replace hyphens with space")
getAvgTxtLen(earnings_transcript_df)

# remove non alphanumeric characters
earnings_transcript_df['Summary'] = earnings_transcript_df['Summary'].str.replace('[^a-zA-Z0-9 ]', ' ')
earnings_transcript_df['Q&A'] = earnings_transcript_df['Q&A'].str.replace('[^a-zA-Z0-9 ]', ' ')
print("remove non alphabetical characters")
getAvgTxtLen(earnings_transcript_df)


Remove URLS
16292.77659574468
28507.675531914894
remove HTML tags
16292.771276595744
28088.38829787234
remove extra whitespaces
16292.771276595744
28088.38829787234
remove contractions
16066.127659574468
27840.452127659573
Remove characters following digits such as 1st, 2nd or 3rd
16066.127659574468
27840.452127659573
Replace hyphens with space
16066.127659574468
27840.452127659573
remove non alphabetical characters
16066.127659574468
27840.452127659573


In [12]:
earnings_transcript_df.iloc[0,2]

'Apple'

In [16]:
stop_words = stopwords.words('english')
additional_stop_words= ['good', 'welcome','also', 'data', 'across', 'google', 'see', 'learn', 'website', 'co', 'iphone', 'devices', 'windows', 'generation', 'every', 'gpu', 'everyone', 'create','office', 'server', 'give', 'around', 'relate', 'saw', 'press', 'applications', 'available','software', 'systems', 'youtube', 'microsoft', 'nanometer', 'search', 'please',  'partner', 'helf', 'game', 'use', 'thank', 'apple', 'nand', 'sequentially','pc', 'would', 'azure', 'information',  'work', 'reflect', 'mobile', 'prior', 'platform', 'security', 'conference', 'ago', 'focus', 'mix', '2018', '2017', '2016', 'come', 'experience', 'solutions', 'memory','dram', 'let', 'primarily'  'remain', 'look', 'technology', 'compute', 'help', 'ai', 'today','us', 'like', 'think', 'see', 'go', 'question', 'would', 'us', 'look', 'say', 'come', 'really', 'get', 'thank', 'talk', 'also', 'yes', 'give', 'make', 'bite', 'could', 'take', 'want', 'kind', 'line', 'know', 'back', 'center', 'around', 'end', 'still', 'mean', 'actually', 'way', 'side', 'today', 'use', 'part', 'overall', 'need', 'sure', 'obviously', 'increase', 'follow', 'focus', 'level', 'company', 'game', 'base', 'sort', 'help', 'mix', 'mention', 'comment', 'different', 'euv', 'let', 'even', 'course', 'dram', 'across', 'move', 'compute', 'nanometer', 'learn', 'share', 'important', 'guess', 'believe', 'feel', 'pretty', 'world', 'couple', 'understand', 'saw', 'may', 'china', 'wonder', 'versus', 'try', 'memory', 'many', 'please', 'happen', 'quite', 'prime', 'trend', 'whether', 'basis', 'capacity', 'nand', 'view', 'tool', 'model', 'software', 'probably', 'guy', 'put', 'able', 'ask', 'team', 'thing', 'plan', 'certainly', 'something', 'network', 'day', 'okay', 'server', 'order', 'guide', 'relate', 'early', 'every', 'gpu', 'capex', 'platform', 'earlier', 'provide', '2019', 'areas', 'always', 'mobile', 'bring', 'reason', 'third', 'transition', 'experience', 'applications', 'excite', 'might', 'fact', 'position', 'do', 'ago', 'azure', 'color', 'environment', 'pc', 'play', 'past', 'process', 'partner', 'iphone', 'answer', 'months', 'future', 'curious', 'google', 'within', 'lead', 'sense', '2018', 'include', 'relative', 'longer', 'program', 'support', 'tell', 'strength', 'content', 'layer', 'already', 'hear', 'machine', 'seem', 'search', 'anything', 'target', 'mark', 'helpful', 'channel', 'systems', 'architecture', 'offer', 'factor', 'example', 'particularly', 'design', 'clearly', 'place', 'keep', 'graphics', 'node', 'ability', 'open', 'users', 'youtube', 'create', 'become', 'particular', '2017', 'real', 'issue', 'perspective', 'interest', 'area', 'another', 'case', 'amount', 'amazon', 'aws', 'less', 'announce', 'several', 'fourth', 'generation', 'key', 'gpus', 'video', 'apple', 'platforms', 'upgrade', 'type', 'show', 'deal', 'store', 'clear', 'set', 'especially', 'map', 'update', 'kelly', 'compare', 'buy', 'beyond', 'logic', 'step', 'unit', 'security', 'basically', 'everything', 'r', 'remain', 'approach', 'specific', 'standpoint', 'sundar', 'definitely', 'far', 'together', 'assume', '2016', 'devices', 'custom', 'piece', 'adoption', 'prepare', 'front', '365', 'ryzen', 'digit', 'yet', 'sequentially', 'expectations', 'perhaps', 'india', 'brian', 'remark', 'days', '3d', 'general', '18', 'conclude', 'asps']
stop_words.extend(additional_stop_words)
#print(stop_words)
earnings_transcript_df['Summary'] = earnings_transcript_df['Summary'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
earnings_transcript_df['Summary'] = earnings_transcript_df['Summary'].astype('string')

earnings_transcript_df['Q&A'] = earnings_transcript_df['Q&A'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words) ]))
earnings_transcript_df['Q&A'] = earnings_transcript_df['Q&A'].astype('string')

print("Stopwords")
getAvgTxtLen(earnings_transcript_df)

preprocessed_summary = ""

for summary in earnings_transcript_df["Summary"]:
    preprocessed_summary += summary

preprocessed_qna = ""

for qna in earnings_transcript_df["Q&A"]:
    preprocessed_qna += qna



# Lemmatize the Dataset
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_review(text):
    return ' '.join(lemmatizer.lemmatize(w,pos="v") for w in w_tokenizer.tokenize(text))

earnings_transcript_df['Summary'] = earnings_transcript_df['Summary'].apply(lemmatize_review)
earnings_transcript_df['Summary'] = earnings_transcript_df['Summary'].astype('string')
earnings_transcript_df['Q&A'] = earnings_transcript_df['Q&A'].apply(lemmatize_review)
earnings_transcript_df['Q&A'] = earnings_transcript_df['Q&A'].astype('string')
print("Lemmatize")
getAvgTxtLen(earnings_transcript_df)



Stopwords
8878.675531914894
10111.771276595744
Lemmatize
8878.595744680852
10111.670212765957


In [17]:
avg_length_summary_after_preprocessing = str(earnings_transcript_df['Summary'].str.len().mean()) 
avg_length_qna_after_preprocessing = str(earnings_transcript_df['Q&A'].str.len().mean()) 
print(avg_length_summary_after_preprocessing)
print(avg_length_qna_after_preprocessing)

8878.595744680852
10111.670212765957


In [18]:
earnings_transcript_df

,Quarter,Year,Company,Summary,Q&A
0,Q2,2016,Apple,incorporate second quarter fy16 earn release c...,first clarification term context 2 billion inv...
1,Q1,2016,Apple,1q16 business review c 1 highlight 1 time reco...,hello much term march quarter guidance imply d...
2,Q3,2016,Apple,annotation n p 1 detail 1 instal purchase metr...,much tim little thoughts investments didi inve...
3,Q4,2016,Apple,4q16 business review c 1 highlight 1 result se...,afternoon congratulations tim month 7 measurab...
4,Q3,2017,Apple,3q17 review c 1 highlight 1 revenue grow produ...,luca first gross margin guidance strong tick j...
...,...,...,...,...,...
183,Q1,2020,NVIDIA,turn call simona jankowski investor relations ...,colette little discussion exactly segment term...
184,Q3,2020,NVIDIA,turn call simona jankowski vice president inve...,first one strong sequential growth q4 jensen d...
185,Q2,2021,NVIDIA,simona jankowski begin afternoon nvidia call s...,congratulations strong growth execution jensen...
186,Q4,2020,NVIDIA,afternoon nvidia call quarter fiscal 2020 call...,colette jensen speak drive upside quarter infe...


In [19]:
# Running augmentations
augdf = earnings_transcript_df.copy()
aug = nlpaw.SynonymAug(aug_src='wordnet', model_path=None, name='Synonym_Aug', aug_min=10, aug_max=60, aug_p=0.3)
for i in range(len(augdf)):
    k = aug.augment(earnings_transcript_df.iloc[i]['Summary'],n=1)
    augdf['Summary'][i] = k[0]
    k = aug.augment(earnings_transcript_df.iloc[i]['Q&A'],n=1)
    augdf['Q&A'][i] = k[0]

earnings_transcript_df = earnings_transcript_df.append(augdf)
earnings_transcript_df = earnings_transcript_df.reset_index().drop(columns = ['index'])

In [20]:
earnings_transcript_df

,Quarter,Year,Company,Summary,Q&A
0,Q2,2016,Apple,incorporate second quarter fy16 earn release c...,first clarification term context 2 billion inv...
1,Q1,2016,Apple,1q16 business review c 1 highlight 1 time reco...,hello much term march quarter guidance imply d...
2,Q3,2016,Apple,annotation n p 1 detail 1 instal purchase metr...,much tim little thoughts investments didi inve...
3,Q4,2016,Apple,4q16 business review c 1 highlight 1 result se...,afternoon congratulations tim month 7 measurab...
4,Q3,2017,Apple,3q17 review c 1 highlight 1 revenue grow produ...,luca first gross margin guidance strong tick j...
...,...,...,...,...,...
371,Q1,2020,NVIDIA,turn call simona jankowski investor relations ...,colette little discussion exactly segment term...
372,Q3,2020,NVIDIA,turn call simona jankowski vice president inve...,first one strong sequential growth q4 jensen d...
373,Q2,2021,NVIDIA,simona jankowski begin afternoon nvidia call s...,congratulations strong growth execution jensen...
374,Q4,2020,NVIDIA,afternoon nvidia call quarter fiscal 2020 call...,colette jensen speak drive upside quarter infe...


In [47]:
today = pd.to_datetime("today")
t_days_after = 7
t_days_before = 7
eps_perc_threshold_for_sentiment = 0.15
price_change_perc_threshold_for_sentiment = 3.0
# Consolidated Dataframe with sentiments for all companies
finance_df = pd.DataFrame(columns=["Company", "date", "Quarter", "Year", "percent change (between t-2 and t+2)","EPS Surprise", "Sentiment_mkt_price", "Sentiment_EPS_Surprise"])

company_tickers = ['NVDA','MU','INTC','GOOGL','CSCO','ASML','AMZN','AMD','AAPL','MSFT']



for ticker in company_tickers:
    
    # Get List of dates for Earnings call
    ticker_data = yf.Ticker(ticker)
    ticker_data.earnings_dates.reset_index(inplace=True)
    
    #Format Date column
    ticker_data.earnings_dates["Earnings Date"] = ticker_data.earnings_dates["Earnings Date"].apply(lambda x: x.strftime('%Y-%m-%d'))
    
    # Filter dates within window of our Transcripts
    earnings_dates_df = ticker_data.earnings_dates[(ticker_data.earnings_dates["Earnings Date"]<=today.strftime('%Y-%m-%d')) & (ticker_data.earnings_dates["Earnings Date"]>="2015-12-01")]
    earnings_dates_df.loc[:,"Surprise(%)"]= earnings_dates_df["Surprise(%)"].apply(lambda x: x*100)
    earning_call_dates= earnings_dates_df.iloc[:,0]
    earnings_call_surprise = earnings_dates_df.iloc[:,[0,-1]]
    #print(ticker)
    #print(earnings_call_surprise)
    
    prepared_df = pd.DataFrame(columns=["Company", "date", "Quarter", "Year", "percent change (between t-2 and t+2)","EPS Surprise", "Sentiment_mkt_price","Sentiment_EPS_Surprise"])
    
    historical_data = yf.download(ticker, start="2015-11-01", end="2022-11-29", interval="1d")

    historical_data.reset_index(inplace=True)

    for date in earning_call_dates:
        # Compute Quarters for each date to join finance df with Transcripts DF created in above cells
        month=pd.to_datetime(date).month
        year=pd.to_datetime(date).year
        if month<=3:
            quarter="Q1"
        elif month<=6:
            quarter="Q2"
        elif month<=9:
            quarter="Q3"
        else:
            quarter="Q4"
                
        # Compute % change by looking at T-2 and T+2 window from earnings call date
        # i.e. if Earnings call is 27 Oct 2022 => compute:
        # => (Closing_price['29 Oct'] - Closing_price['27 Oct']) * 100 / (Closing_price['27 Oct'])
        
        index= historical_data.index[historical_data["Date"]==pd.to_datetime(date)]
        closing_price_after_earnings_call = float(historical_data.loc[index+t_days_after,"Adj Close"])
        closing_price_before_earnings_call = float(historical_data.loc[index-t_days_before,"Adj Close"])
        change = (closing_price_after_earnings_call - closing_price_before_earnings_call)*100/closing_price_before_earnings_call
        eps_surprise = float(earnings_call_surprise[earnings_call_surprise["Earnings Date"] == date]["Surprise(%)"])
        #print(earnings_call_surprise[earnings_call_surprise["Earnings Date"] == date, "Surprise(%)"])
        # Computer sentiment based on price change percentage threshold
        if abs(change)<= price_change_perc_threshold_for_sentiment:
            label_mkt_price="Neutral"
        elif change> price_change_perc_threshold_for_sentiment:
            label_mkt_price="Positive"
        else:
            label_mkt_price="Negative"
            
        if abs(eps_surprise - 1.0)<= eps_perc_threshold_for_sentiment:
            label_eps="Neutral"
        elif (eps_surprise)> 1.0 + eps_perc_threshold_for_sentiment:
            label_eps="Positive"
        else:
            label_eps="Negative"   
        prepared_df = prepared_df.append({"Company":companies[ticker],"date":date,"Quarter":quarter, "Year": year, "percent change (between t-2 and t+2)": change,"EPS Surprise": eps_surprise, "Sentiment_mkt_price": label_mkt_price, "Sentiment_EPS_Surprise":label_eps}, ignore_index=True)
    prepared_df.reset_index(drop=True)
    prepared_df.to_csv("..\\data\\OHLC\\"+ticker+".csv", encoding='utf-8')
    finance_df=pd.concat([finance_df, prepared_df], ignore_index=True)
finance_df

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Company,date,Quarter,Year,percent change (between t-2 and t+2),EPS Surprise,Sentiment_mkt_price,Sentiment_EPS_Surprise
0,NVIDIA,2022-11-16,Q4,2022,10.670590,-16.43,Positive,Negative
1,NVIDIA,2022-08-24,Q3,2022,-28.294456,-59.36,Negative,Negative
2,NVIDIA,2022-05-25,Q2,2022,8.816044,5.18,Positive,Positive
3,NVIDIA,2022-02-16,Q1,2022,-1.387086,8.02,Neutral,Positive
4,NVIDIA,2021-11-17,Q4,2021,8.349566,5.79,Positive,Positive
...,...,...,...,...,...,...,...,...
275,Microsoft,2017-01-26,Q1,2017,1.775153,4.93,Neutral,Positive
276,Microsoft,2016-10-20,Q4,2016,4.773554,11.27,Positive,Positive
277,Microsoft,2016-07-19,Q3,2016,7.476121,18.76,Positive,Positive
278,Microsoft,2016-04-21,Q2,2016,-7.392503,-2.67,Negative,Negative


In [50]:
finance_df.Year = finance_df.Year.astype('int')
earnings_transcript_df.Year = earnings_transcript_df.Year.astype('int')

In [51]:
df_merged = pd.merge(earnings_transcript_df, finance_df, on=['Company', 'Year', 'Quarter'], how='inner')

In [52]:
df_merged

,Quarter,Year,Company,Summary,Q&A,date,percent change (between t-2 and t+2),EPS Surprise,Sentiment_mkt_price,Sentiment_EPS_Surprise
0,Q2,2016,Apple,incorporate second quarter fy16 earn release c...,first clarification term context 2 billion inv...,2016-04-26,-14.603822,-4.90,Negative,Negative
1,Q2,2016,Apple,incorporate second quarter fy16 earn release o...,first clarification term context 2 billion inv...,2016-04-26,-14.603822,-4.90,Negative,Negative
2,Q1,2016,Apple,1q16 business review c 1 highlight 1 time reco...,hello much term march quarter guidance imply d...,2016-01-26,-2.407392,1.64,Neutral,Positive
3,Q1,2016,Apple,1q16 business review c 1 highlight 1 time reco...,hello much term march quarter guidance imply d...,2016-01-26,-2.407392,1.64,Neutral,Positive
4,Q3,2016,Apple,annotation n p 1 detail 1 instal purchase metr...,much tim little thoughts investments didi inve...,2016-07-26,7.758173,2.60,Positive,Positive
...,...,...,...,...,...,...,...,...,...,...
361,Q2,2021,NVIDIA,simona jankowski begin afternoon nvidia call s...,congratulations strong growth execution jensen...,2021-05-26,24.379663,11.45,Positive,Positive
362,Q4,2020,NVIDIA,afternoon nvidia call quarter fiscal 2020 call...,colette jensen speak drive upside quarter infe...,2020-11-18,-1.681848,13.22,Neutral,Positive
363,Q4,2020,NVIDIA,afternoon nvidia call quarter fiscal 2020 call...,colette jensen speak drive upside quarter infe...,2020-11-18,-1.681848,13.22,Neutral,Positive
364,Q1,2021,NVIDIA,simona jankowski begin afternoon nvidia call f...,congratulations solid quarter colette commenta...,2021-02-24,-16.708161,10.40,Negative,Positive


In [106]:
train_x_y = df_merged[['Summary','Sentiment_EPS_Surprise']].sample(frac = 0.80, random_state = 42)
test_x_y = pd.concat([df_merged[['Summary','Sentiment_EPS_Surprise']], train_x_y]).drop_duplicates(keep=False)
train_x_y["Sentiment_EPS_Surprise"] = le.fit_transform(train_x_y["Sentiment_EPS_Surprise"])
test_x_y["Sentiment_EPS_Surprise"] = le.fit_transform(test_x_y["Sentiment_EPS_Surprise"])

In [108]:
X = df_merged[['Summary']]
Y = df_merged[['Sentiment_EPS_Surprise']]
Y

,Sentiment_EPS_Surprise
0,Negative
1,Negative
2,Positive
3,Positive
4,Positive
...,...
361,Positive
362,Positive
363,Positive
364,Positive


In [113]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42, shuffle=True, stratify=Y)
train_x_y = pd.concat([x_train, y_train], axis=1,  names=['Summary', 'Sentiment_EPS_Surprise'])
test_x_y = pd.concat([x_test, y_test], axis=1, names=['Summary', 'Sentiment_EPS_Surprise'])

In [114]:
print(train_x_y.shape)
print(test_x_y.shape)

(245, 2)
(121, 2)


In [115]:
test_x_y

,Summary,Sentiment_EPS_Surprise
164,introduce marilyn mora head investor relations...,Neutral
139,technical trouble veldhoven netherlands asml c...,Positive
132,turn call mr skip miller ahead sir skip mornin...,Positive
88,turn call director investor relations dave fil...,Positive
334,turn call arnab chanda vice president investor...,Positive
...,...,...
288,turn host mr mike spencer manager investor rel...,Positive
115,turn mr craig deyoung ahead sir patricia after...,Positive
217,hand loudspeaker system ellen west head invest...,Negative
200,turn ellen west head investor relations ahead ...,Positive


In [116]:
train_x_y["Sentiment_EPS_Surprise"] = le.fit_transform(train_x_y["Sentiment_EPS_Surprise"])
test_x_y["Sentiment_EPS_Surprise"] = le.fit_transform(test_x_y["Sentiment_EPS_Surprise"])

In [117]:
train_x_y

,Summary,Sentiment_EPS_Surprise
183,michelle cisco quarter fiscal 2020 quarterly e...,2
192,ladies gentlemen alphabet quarter earn call af...,2
303,turn call ms shanye hudson madam begin chelsea...,2
33,2q20 review c 1 overview 1 revenue 58 3b 1 tim...,2
140,turn call mr skip miller ahead sir operator sk...,2
...,...,...
103,hello q3 financial result shout join olsavsky ...,0
282,afternoon join call satya nadella chief execut...,2
346,turn call simona jankowski vice president inve...,0
99,hello q4 fiscal consequence call option join o...,2


In [118]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

model_args = ClassificationArgs()
model_args.train_batch_size = 2
model_args.gradient_accumulation_steps = 10
model_args.learning_rate = 3e-4
model_args.num_train_epochs = 6
model_args.overwrite_output_dir=True
model_bert = ClassificationModel("bert", "bert-base-uncased", num_labels=3, args=model_args, use_cuda=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [119]:
model_bert.train_model(train_x_y, overwrite_output_dir=True)

  0%|          | 0/245 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/123 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/123 [00:00<?, ?it/s]

(30, 0.6761158640806874)

In [121]:
pred_bert, out_bert = model_bert.predict(list(test_x_y['Summary'].values))

acc_bert = accuracy_score(test_x_y['Sentiment_EPS_Surprise'].to_numpy(), pred_bert)
f1_bert = f1_score(test_x_y['Sentiment_EPS_Surprise'].to_numpy(), pred_bert, average='micro')

print("Accuracy score -->", acc_bert)
print("F1 score -->", f1_bert)

  0%|          | 0/121 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

Accuracy score --> 0.8181818181818182
F1 score --> 0.8181818181818182


In [ ]:
pred_bert

In [123]:
test_x_y['Sentiment_EPS_Surprise']

164    1
139    2
132    2
88     2
334    2
      ..
288    2
115    2
217    0
200    2
220    2
Name: Sentiment_EPS_Surprise, Length: 121, dtype: int32

In [125]:
precision = precision_score(pred_bert, test_x_y['Sentiment_EPS_Surprise'].to_numpy(), average='micro')
precision

0.8181818181818182

In [126]:
recall = recall_score(pred_bert, test_x_y['Sentiment_EPS_Surprise'].to_numpy(), average='micro')

In [128]:
recall

0.8181818181818182

## REFERENCES
https://www.kaggle.com/code/kamalkhumar/financial-news-analysis-using-bert-with-eda